In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("data"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1)

# Any results you write to the current directory are saved as output.

2024-05-30 02:40:14.628064: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 02:40:14.628214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 02:40:14.788096: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 02:40:15.020081: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 02:40:16.029414: W tensorflow/compiler/tf2

['Hindi_English_Truncated_Corpus.csv', 'images', 'pneumonia']


In [2]:
lines=pd.read_csv("data/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [3]:
lines['source'].value_counts()

source
tides        50000
ted          39881
indic2012    37726
Name: count, dtype: int64

In [4]:
#lines=lines[lines['source']=='ted']

In [5]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,"The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .","कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी ."
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
9,tides,You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .,"हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . ."


In [6]:
pd.isnull(lines).sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [7]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [8]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [9]:
lines=lines.sample(n=60000,random_state=42)
lines.shape

(60000, 3)

In [10]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [11]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [12]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [13]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [14]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
lines.head()

,source,english_sentence,hindi_sentence
25520,indic2012,islam is word from arabic and it full word is salamaits definition peace surrender,START_ इस्लाम शब्द अरबी भाषा का शब्द है जिसका मूल शब्द सल्लमा है जिस की दो परिभाषाएं हैं शान्ति आत्मसमर्पण। _END
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END
111804,tides,the provision would not affect the power of parliament to make laws in respect of income from professions etc lrb article rrb,START_ व्यवसायों आदि से होने वाली आय के बारे में विधि बनाने की संसद की शक्ति पर उपबंध का प्रभाव नहीं पड़ेगा अनुच्छेद _END


In [16]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [17]:
len(all_eng_words)

49864

In [18]:
all_eng_words

{'rehengold',
 'stewart',
 'onionshaped',
 'besind',
 'legislatively',
 'orwellian',
 'rogue',
 'fidelity',
 'defamatory',
 'shakti',
 'interim',
 'accessed',
 'tek',
 'antiban',
 'proffesordirector',
 'singaldib',
 'dreaded',
 'payyanur',
 'mahendranagar',
 'brook',
 'reminded',
 'pundit',
 'imprisoned',
 'mukhabhigaman',
 'sullivan',
 'hurdles',
 'sheath',
 'troika',
 'contending',
 'customised',
 'matchesevery',
 'vanaprasta',
 'abut',
 'iswara',
 '“”primitive',
 'categorygateway',
 'longstanding',
 'haward',
 'virendranagar',
 'flows',
 'compoany',
 'premodern',
 'provided',
 'flux',
 'blackmail',
 'puxi',
 'posterity',
 'increase”',
 '“lauren',
 'bowler',
 'rick',
 'minimized',
 'abyss',
 'anthve',
 'stadiums',
 'vientiane',
 'parisodh',
 'solamorales',
 'percent',
 'gaiments',
 'airports',
 'abscence',
 'deference',
 'deceive',
 'jasop',
 'underside',
 'houses',
 'abitabhs',
 'foetal',
 'barriers',
 'insect',
 'animate',
 'reciting',
 'rhetenor',
 'hers',
 'multipurpose',
 '“”obs

In [19]:
len(all_hindi_words)

57776

In [20]:
all_hindi_words

{'पांरपरिक',
 'पिज़्ज़ा',
 'संग्रहालय',
 'फास्फेट',
 'रोमांस',
 'नथुनों',
 'आतऋऊण्श्छ्ष्मविशऋऊण्श्छ्ष्वास',
 'विशेषणों',
 'घटकों',
 'खुर्ची',
 'चाटनी',
 'वेबकूफी',
 'शान्तनु',
 'अपरोधों',
 'सीएच',
 '“किसका',
 'अर्ध',
 'ओबामा',
 'परिमल',
 'बड़े।',
 'अर्थव्यवस्थाओं',
 'स्वेच्छया',
 'छपने',
 'आश्यक',
 'dmzla',
 'संवेदनशीलता',
 'चोली',
 'फायदा',
 'मैट्रो',
 'परिव्याप्त',
 'समानांतर',
 'गढै',
 'हिमालकी',
 'मरता',
 'कहानियो',
 'पोजीशनः',
 'शब्दराशि',
 'स्टेट्स',
 'आज्ञावली',
 'पंखुडियां',
 'प्रतिबिंब',
 'अ',
 'वसव',
 'doaa',
 'आयाताकार',
 'अनिर्भर',
 'जायगा',
 'एक्सेलटर',
 'दरिया',
 'शोएब',
 'न्युन्देन',
 'मुलऋऊण्श्छ्ष्कों',
 'समप्त',
 'अंगुलियाँ',
 'तिलहनों',
 'बेरोकटोक',
 'दिन',
 'छरे',
 'अनऋऊण्श्छ्ष्याय',
 'गठित',
 'उपलपब्ध',
 'वंदना',
 'ट्राम',
 'बसें।',
 'फेरुनुलम',
 'स्कॉटलंड',
 'कदर',
 'पुश्त',
 'rick',
 'सीतारामैया',
 'नामॊं',
 'शनिवार',
 'श्रेणीऊर्जा',
 'डारपा',
 'वोई',
 'खेदजनक',
 'बनावट',
 'सर्वप्रथम',
 'शीघवांग',
 'मॉंगकर',
 'पर्चियों',
 'गतिवरोध',
 'अवदान',
 'तूफ़ान',
 'सलहमशविर

In [21]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [22]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
25520,indic2012,islam is word from arabic and it full word is salamaits definition peace surrender,START_ इस्लाम शब्द अरबी भाषा का शब्द है जिसका मूल शब्द सल्लमा है जिस की दो परिभाषाएं हैं शान्ति आत्मसमर्पण। _END,14,21
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END,7,9
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END,6,9
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END,4,6
111804,tides,the provision would not affect the power of parliament to make laws in respect of income from professions etc lrb article rrb,START_ व्यवसायों आदि से होने वाली आय के बारे में विधि बनाने की संसद की शक्ति पर उपबंध का प्रभाव नहीं पड़ेगा अनुच्छेद _END,22,24


In [23]:
lines[lines['length_eng_sentence']>30].shape

(5890, 5)

In [24]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [25]:
lines

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END,7,9
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END,6,9
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END,4,6
57202,ted,there was lasagna there was casseroles,START_ वहां लाजान्या था कैसेरोल थे _END,6,7
107821,indic2012,super power india source google writer vedpratap vedik,START_ महाशक्ति भारत गूगल पुस्तक लेखक वेदप्रताप वैदिक _END,8,9
...,...,...,...,...,...
118420,tides,avoid excess quantity of salt and sugar,START_ अधिक नमक और शक्कर वाले खाद्य पदार्थो को खाने से बचे _END,7,13
96611,indic2012,it was took twelve years to built foundation and mausoleum,START_ आधारशिला एवं मकबरे को निर्मित होने में बारह साल लगे। _END,10,12
30109,indic2012,it is an upanishad given under “”bhishmaparva“” section of mahabharat composition,START_ यह एक महाभारत के भीष्मपर्व के अन्तर्गत दिया गया एक उपनिषद है । _END,11,15
126495,ted,“i love the shoes but i hate the laces”,START_ “मुझे जूते अच्छे लगे पर इसके फीते अच्छे नहीं हैं ” _END,9,13


In [26]:
lines.shape

(39594, 5)

In [27]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [28]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [29]:
print(max_length_src)

20


In [30]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(49864, 57776)

In [31]:
num_decoder_tokens += 1 #for zero padding


In [32]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [33]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [34]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
49098,ted,a governor of one of the oilrich states of nigeria,START_ नाइजीरिया के तेल समृद्ध राज्यों के राज्यपाल _END,10,9
64692,indic2012,vishwanathan anand who hails from the country which gave birth to chess india has performed well too,START_ शतरंज के जनक देश भारत के खिलाड़ी विश्वनाथ आनंद ने अच्छा प्रदर्शन किया है । _END,17,17
94614,ted,somehow right in the user interface,START_ मुझे वहीं साइट पर ही _END,6,7
20734,tides,the milkyield is very low,START_ इनसे दूध बहुत कम मिलता है _END,5,8
120600,ted,this was a prison assignment which was one big den of criminals,START_ ये एक जेल का कार्य था जिसमे बहुत बड़ा समूह था मुजरिमों का _END,12,15
36523,indic2012,jaisamare,START_ जैसलमेर _END,1,3
63221,tides,it is twostoreyed with pent and gable roof patterns for its storeys,START_ यह दो मंजिला है और इसकी मंजिलों पर ढलवां और त्रिअंकी प्रकार की छत है _END,12,17
2123,ted,need more resources and need our help,START_ उन्हें साधन और हमारी सहायता की जरुरत है _END,7,10
26912,tides,unfortunately no agreement or settlement was possible,START_ दुर्भाग़्य से कोई समझौता या निपटारा संभव नहीं हुआ _END,7,11
97258,tides,critics demand justification for the use of resources to benefit a few,START_ आलचक थोड़ैसे लगों के लभ के लिए संसाधनों के इस्तेमाल पर सवाल उ आते हैं _END,12,17


### Split the data into train and test

In [35]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((31675,), (7919,))

In [36]:
X_train

22245     i protection of the interests of artisans                                                   
34551     and what is more important because we want to get away from oil                             
123862    man i feel her                                                                              
62143     now theyre all people weve investigated                                                     
97740     study of quran deniel pipes                                                                 
                     ...                                                                              
63458     padma purana                                                                                
11216     while all these changes sound phenomenal the teething troubles are also beginning to show up
76796     helps push the boundaries of possibility                                                    
22346     surawali                                                       

In [37]:
y_train

22245     START_ शिल्पियों के हितों का संरक्षण _END                                                                  
34551     START_ और अधिक महत्वपूर्ण क्या है क्योंकि हम तेल से आगे जाना चाहते थे _END                                 
123862    START_ यार मैं उसे महसूस कर सकता हु _END                                                                   
62143     START_ ये सभी वो लोग हैं जिनकी हमने जांच करी थी _END                                                       
97740     START_ कुरान का अध्ययन डेनियल पाइप्स _END                                                                  
                            ...                                                                                      
63458     START_ पद्म पुराण _END                                                                                     
11216     START_ हालंकि ये परिवर्तन चमत्कारी लगते हैं लेकिन शुरू में पेश आने वाली समस्याएं भी जाहिर होने लगी हैं _END
76796     START_ संभावना की सीमाओं को आगे बढाती है _END 

### Let us save this data

In [38]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [39]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [40]:
latent_dim=300

In [2]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

NameError: name 'num_encoder_tokens' is not defined

In [42]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [43]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [44]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 300)            1495920   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            1733310   ['input_2[0][0]']         

In [45]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 80

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



/tmp/ipykernel_72925/2713035456.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/80


2024-05-29 08:27:34.322363: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_36/output/_23'
2024-05-29 08:27:34.749880: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-05-29 08:27:34.982300: I external/local_xla/xla/service/service.cc:168] XLA service 0x724be51ffb80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-29 08:27

494/494 [==============================] - 202s 401ms/step - loss: 7.3369 - val_loss: 7.0358
Epoch 2/80
494/494 [==============================] - 196s 397ms/step - loss: 6.9882 - val_loss: 7.0094
Epoch 3/80
494/494 [==============================] - 194s 394ms/step - loss: 6.9529 - val_loss: 6.9781
Epoch 4/80
494/494 [==============================] - 191s 387ms/step - loss: 6.9246 - val_loss: 6.9556
Epoch 5/80
494/494 [==============================] - 190s 385ms/step - loss: 6.8765 - val_loss: 6.8912
Epoch 6/80
494/494 [==============================] - 192s 388ms/step - loss: 6.7730 - val_loss: 6.7712
Epoch 7/80
494/494 [==============================] - 191s 387ms/step - loss: 6.6502 - val_loss: 6.6684
Epoch 8/80
494/494 [==============================] - 191s 386ms/step - loss: 6.5392 - val_loss: 6.5797
Epoch 9/80
494/494 [==============================] - 191s 387ms/step - loss: 6.4508 - val_loss: 6.5110
Epoch 10/80
494/494 [==============================] - 191s 387ms/step - lo

In [ ]:
model.save_weights('nmt_weights.h5')


In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
model.save('models/english_to_hindi_translator.h5')